<a href="https://colab.research.google.com/github/barkhaaroraa/nlp_or_ai-agents/blob/main/qdrant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q qdrant-client sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 42.6 MB/s eta 0:00:00


In [4]:
!pip install hf_xet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 MB 15.6 MB/s eta 0:00:00


In [5]:
from sentence_transformers import SentenceTransformer

# Load a small, efficient model
model = SentenceTransformer('all-MiniLM-L6-v2')

In [6]:
from qdrant_client import QdrantClient

qdrant_client = QdrantClient(
    url="",
    api_key="",
)

print(qdrant_client.get_collections())

collections=[]


In [7]:
from qdrant_client.models import VectorParams, Distance

In [12]:
qdrant_client.create_collection(
    collection_name="agent_memoryy",
    vectors_config=VectorParams(size=384, distance=Distance.COSINE)
)


UnexpectedResponse: Unexpected Response: 409 (Conflict)
Raw response content:
b'{"status":{"error":"Wrong input: Collection `agent_memoryy` already exists!"},"time":0.022444195}'

In [13]:
from uuid import uuid4

def embed_text(text):
    return model.encode(text).tolist()

def store_message(text):
    vector = embed_text(text)
    payload = {"message": text}
    qdrant_client.upsert(
        collection_name="agent_memory",
        points=[{
            "id": str(uuid4()),     # unique ID
            "vector": vector,       # the embedding
            "payload": payload      # original text stored as metadata
        }]
    )


In [17]:
def search_similar(query, top_k=5):
    vector = embed_text(query)
    results = qdrant_client.search(
        collection_name="agent_memory",
        query_vector=vector,
        limit=top_k
    )
    return [hit.payload["message"] for hit in results]


In [ ]:
def run_chat():
    print("💬 Type a message. Type 'exit' to stop.")
    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit"]:
            break

        print("\n🧠 Recalling similar memories...")
        similar = search_similar(user_input)
        if similar:
            for i, msg in enumerate(similar, 1):
                print(f"{i}. {msg}")
        else:
            print("No similar messages found.")

        print("✅ Storing your message...\n")
        store_message(user_input)

run_chat()


💬 Type a message. Type 'exit' to stop.
You: i love italian food

🧠 Recalling similar memories...


<ipython-input-17-1e68a21e8f23>:3: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant_client.search(


1. pizza
2. pizza toppings with pasta
3. pizza sauce
4. pasta is white sauce
5. whats pasta
✅ Storing your message...

You: i love indian food

🧠 Recalling similar memories...
1. i love italian food
2. tell me everything baou pasta
3. pasta is white sauce
4. white sauce pasta has cheese in it
5. pizza
✅ Storing your message...

You: mountains ranges are beautiful

🧠 Recalling similar memories...
1. i love indian food
2. i love italian food
3. hello world
4. pizza toppings with pasta
5. hello
✅ Storing your message...

You: i love hiking

🧠 Recalling similar memories...
1. mountains ranges are beautiful
2. i love indian food
3. i love italian food
4. hello
5. pizza
✅ Storing your message...

You: clouds are white while the sky is blue

🧠 Recalling similar memories...
1. pasta is white sauce
2. what is white sauce pasta
3. mountains ranges are beautiful
4. white sauce pasta has cheese in it
5. white sauce pasta has cheese blended sauce
✅ Storing your message...

You: airplane flies at a 